In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
%matplotlib inline

from pprint import pprint
from zen_garden.postprocess.results import Results

ModuleNotFoundError: No module named 'matplotlib'

### Working with results of a ZenGarden run

This notebook shows some examples on how to handle the results of a `zen-garden` run. It uses the outputs of the tests as examples. Therefore, you need to run the tests via 
```
coverage run --source="zen_garden" -m pytest -v tests/testcases/run_test.py
```
to get the data.

### Reading in the results

The results can be read in using the `Results` class. This will automatically read in all relevant files, independent of the number of scenarios or if the optimization was run with perfect or myopic foresight. However, you should know what type of input you are reading in, as the output of some routines will depend on the type.

In [ ]:
# This has no scenarios and perfect foresight
res_basic = Results("../outputs/test_1a/")
# Here we have three different scenarios
res_scenario = Results("../outputs/test_6a/")
# This was run with myopic foresight
res_mf = Results("../outputs/test_4g/")
# This was run with aggregated time steps and multi year
res_ts = Results("../outputs/test_4f/")

### Accessing the config

The standard dictionaries, i.e. system, analysis, scenarios, solver and dict_sequence_time_steps can be accessed via the results dict.

In [ ]:
# Show the analysis settings
pprint(res_basic.results["analysis"])

### Extracting individual data frames

You can access individual data frames from any component with the `get_df` method. However, the behavior differs slightly for the usecases.
If the optimization was run with perfect foresight and no scenarios, then the data frame is returned as `pandas.Series`. For myopic foresight, the final dataframe is stiched together from the relevant outputs if the component is set yearly. If multiple scenarios were run, a dictionary containing the data frame for all different scenarios is returned. However, you can specify the scenario in advance, then only a single data frame is returned.

Additionally, the method provides a shortcut to save the extracted dataframe directly to a CSV, or in case of different scenarios to a bundle of CSV files.

In [ ]:
# This will return a single data frame
df = res_basic.get_df('export_carrier_flow')
df

In [ ]:
# you can also save the dataframe directly to a csv:
df = res_basic.get_df('carbon_emissions_overshoot', to_csv="test")
# this prints out the content of the generated csv
!cat test.csv

In [ ]:
# if we have multiple scenarios, all dataframe will be returned in a dict
df_dict = res_scenario.get_df('export_carrier_flow')
pprint(df_dict)

In [ ]:
# saving this to a csv will create a file for each scenario
df_dict = res_scenario.get_df('export_carrier_flow', to_csv="test2")
# list files and print content of one
!echo "Generated files: "
!ls test2*
!echo 
!echo "Output of test2_scenario_2.csv"
!cat test2_scenario_2.csv

In [ ]:
# we can also specify a specific scneario
df = res_scenario.get_df('export_carrier_flow', scenario="scenario_2")
df

In [ ]:
# This will collect the values from the results folders of the different myopic foresight runs
df = res_mf.get_df("carbon_emissions_cumulative")
df

### Helper functions

The `Results` class implements some helper functions. The function `get_total` allows you to calculate the total Value of a component, for a given year, element and scenario if desired. And the function `get_full_ts` calculates the full (disaggregate) time series for a given component.

In [ ]:
# We read out a normal dataframe
df = res_mf.get_df("input_flow")
print(df)

In [ ]:
# We can calculate the total value of the dataframe
res_mf.get_total(df)

In [ ]:
# and also get the full timeseries (disaggregate) if we want
df_disaggregate = res_mf.get_full_ts(df)
df_disaggregate

### Some Plots

Here could be some example plots